<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Data" data-toc-modified-id="Data-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Data</a></span></li><li><span><a href="#Code" data-toc-modified-id="Code-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Code</a></span><ul class="toc-item"><li><span><a href="#JH" data-toc-modified-id="JH-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>JH</a></span></li></ul></li><li><span><a href="#Train" data-toc-modified-id="Train-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Train</a></span><ul class="toc-item"><li><span><a href="#With-scaling-preds" data-toc-modified-id="With-scaling-preds-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>With scaling preds</a></span></li><li><span><a href="#Without-scaling-preds" data-toc-modified-id="Without-scaling-preds-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Without scaling preds</a></span></li></ul></li><li><span><a href="#Stepping-Through-a-Batch" data-toc-modified-id="Stepping-Through-a-Batch-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Stepping Through a Batch</a></span><ul class="toc-item"><li><span><a href="#JH" data-toc-modified-id="JH-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>JH</a></span><ul class="toc-item"><li><span><a href="#Init" data-toc-modified-id="Init-4.1.1"><span class="toc-item-num">4.1.1&nbsp;&nbsp;</span>Init</a></span></li><li><span><a href="#Get-batch,-acts,-item" data-toc-modified-id="Get-batch,-acts,-item-4.1.2"><span class="toc-item-num">4.1.2&nbsp;&nbsp;</span>Get batch, acts, item</a></span></li><li><span><a href="#ssd-item-loss" data-toc-modified-id="ssd-item-loss-4.1.3"><span class="toc-item-num">4.1.3&nbsp;&nbsp;</span>ssd item loss</a></span></li><li><span><a href="#lbl-loss" data-toc-modified-id="lbl-loss-4.1.4"><span class="toc-item-num">4.1.4&nbsp;&nbsp;</span>lbl loss</a></span></li></ul></li></ul></li></ul></div>

**~ My Code ~**

# Data

In [ ]:
### Imports & Paths ###
from fastai.vision.all import *
import pandas as pd


def random_seed(s, use_cuda):
    #Also, remember to use num_workers=0 when creating the DataBunch
    np.random.seed(s)
    torch.manual_seed(s)
    random.seed(s)
    if use_cuda:
        torch.cuda.manual_seed(s)
        torch.cuda.manual_seed_all(s)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False      
random_seed(42,True)


### Params ###
im_sz   = 224
bs      = 64
val_pct = .2
sub_pct = 1
path = untar_data(URLs.PASCAL_2007)
annos_path = path/'train.json'
ims_path = path/'train'


### Items ###
fns, annos = get_annotations(annos_path)
fn2anno = {f:a for f,a in zip(fns,annos)}
def get_im(f):   return ims_path/f
def get_bbox(f): return fn2anno[f][0]
def get_lbl(f):  return fn2anno[f][1]


### DataLoaders ###
itfms = Resize(im_sz, method='squish')
btfms = setup_aug_tfms([Rotate(), Brightness(), Contrast(), Flip(),
                       Normalize.from_stats(*imagenet_stats)])
db = DataBlock(
    blocks=[ImageBlock, BBoxBlock, BBoxLblBlock(add_na=False)],
    get_x=get_im, get_y=[get_bbox, get_lbl], n_inp=1,
    splitter=RandomSplitter(val_pct),
    item_tfms=itfms, batch_tfms=btfms)
subset = L(fns).shuffle()[0:int(len(fns)*sub_pct)]
# dls = db.dataloaders(subset, bs=bs)
dls = db.dataloaders(fns, bs=bs, seed=42)
dls.v = dls.vocab
dls.ncls = len(dls.vocab)

In [ ]:
### Inspection ###
print("Vocab:", dls.v)
print("Size of train data:",len(dls.train.items))
print("Size of valid data:",len(dls.valid.items))
for i,t in enumerate(dls.one_batch()):
    print(f"batch[{i}]:",'\t',t.dtype,'\t',t.shape)

Vocab: (#20) ['aeroplane','bicycle','bird','boat','bottle','bus','car','cat','chair','cow'...]
Size of train data: 2001
Size of valid data: 500
batch[0]: 	 torch.float32 	 torch.Size([64, 3, 224, 224])
batch[1]: 	 torch.float32 	 torch.Size([64, 12, 4])
batch[2]: 	 torch.int64 	 torch.Size([64, 12])


Interpretation of tensor shapes:
- torch.Size([128, 3, 224, 224]): bs, channels (rgb), im_sz, im_sz
- torch.Size([128, 20, 4]): bs, max objs for a single im in batch, bb coords
- torch.Size([128, 20]): bs, max objs for a single im in batch

# Code

## JH

In [ ]:
### Anchors ###
anc_grids = [4,2,1]
# anc_grids = [4]
anc_zooms = [0.75, 1., 1.3]
# anc_zooms = [1.]
anc_ratios = [(1.,1.), (1.,0.5), (0.5,1.)]
# anc_ratios = [(1.,1.)]
anchor_scales = [(anz*i,anz*j) for anz in anc_zooms for (i,j) in anc_ratios]
k = len(anchor_scales)
anc_offsets = [1/(o*2) for o in anc_grids]
anc_x = np.concatenate([np.tile(np.linspace(ao, 1-ao, ag), ag)
                        for ao,ag in zip(anc_offsets,anc_grids)])
anc_y = np.concatenate([np.repeat(np.linspace(ao, 1-ao, ag), ag)
                        for ao,ag in zip(anc_offsets,anc_grids)])
anc_ctrs = np.repeat(np.stack([anc_x,anc_y], axis=1), k, axis=0)
anc_sizes  =   np.concatenate([np.array([[o/ag,p/ag] for i in range(ag*ag) for o,p in anchor_scales])
               for ag in anc_grids])
grid_sizes = tensor(np.concatenate([np.array([ 1/ag       for i in range(ag*ag) for o,p in anchor_scales])
               for ag in anc_grids]), requires_grad=False).unsqueeze(1)
anchors = tensor(np.concatenate([anc_ctrs, anc_sizes], axis=1), requires_grad=False).float()

def hw2corners(ctr, hw): return torch.cat([ctr-hw/2,ctr+hw/2], dim=1)
anchor_cnr = hw2corners(anchors[:,:2], anchors[:,2:])

In [ ]:
### Architecture ###
class StdConv(nn.Module):
    def __init__(self, n_in, n_out, stride=2, dp=0.1):
        super().__init__()
        self.conv = nn.Conv2d(n_in,n_out,3,stride=stride,padding=1)
        self.bn = nn.BatchNorm2d(n_out)
        self.dropout = nn.Dropout(dp)
        
    def forward(self,x):
        return self.dropout(self.bn(F.relu(self.conv(x))))
    
def flatten_conv(x,k):
    bs,nf,gx,gy = x.size()
    x = x.permute(0,2,3,1).contiguous()
    return x.view(bs,-1,nf//k)

class OutConv(nn.Module):
    def __init__(self, k, n_in, bias):
        super().__init__()
        self.k = k
        self.bbs  = nn.Conv2d(n_in,            4*k, 3, padding=1)
        self.lbls = nn.Conv2d(n_in, (dls.ncls+1)*k, 3, padding=1)
        self.lbls.bias.data.zero_().add_(bias)
        
    def forward(self,x):
        return [flatten_conv(self.bbs(x),  self.k),
                flatten_conv(self.lbls(x), self.k)]
    
class SSD_MultiHead(nn.Module):
    def __init__(self, k, bias=-4., drop=0.4):
        super().__init__()
        self.drop = nn.Dropout(drop)
        self.sconv1 = StdConv(512,256, dp=drop)
        self.sconv2 = StdConv(256,256, dp=drop)
        self.sconv3 = StdConv(256,256, dp=drop)
        self.out1 = OutConv(k, 256, bias)
        self.out2 = OutConv(k, 256, bias)
        self.out3 = OutConv(k, 256, bias)

    def forward(self, x):
        x = self.drop(F.relu(x))
        x = self.sconv1(x)
        bbs1,lbls1 = self.out1(x)
        x = self.sconv2(x)
        bbs2,lbls2 = self.out2(x)
        x = self.sconv3(x)
        bbs3,lbls3 = self.out3(x)
        return [torch.cat([ bbs1, bbs2, bbs3], dim=1),
                torch.cat([lbls1,lbls2,lbls3], dim=1)]
    
class CustMod(Module):
    """A module made from a pretrained body and an untrained head."""
    def __init__(self, body, head):
        self.body, self.head = body, head
        
    def forward(self, x):
        return self.head(self.body(x))

    
    
### FocalLoss ###
def one_hot_embedding(labels, num_classes):
    return torch.eye(num_classes)[labels.data].to(device)

class BCE_Loss(nn.Module):
    def __init__(self, num_classes, device):
        super().__init__()
        self.num_classes, self.device = num_classes, device
    
    def forward(self, preds, targets):
        t = one_hot_embedding(targets, self.num_classes+1)
        t = tensor(t[:,:-1].contiguous())
        x = preds[:,:-1]
        w = self.get_weight(x,t).detach()
        return F.binary_cross_entropy_with_logits(x, t, w, reduction='sum') / self.num_classes
    
    def get_weight(self,x,t):
        return None

class FocalLoss(BCE_Loss):
    def get_weight(self,x,t):
        alpha,gamma = 0.25,2.
        p = x.sigmoid()
        pt = p*t + (1-p)*(1-t)
        w = alpha*t + (1-alpha)*(1-t)
        return w * (1-pt).pow(gamma)

    
### IoU ###
def intersection(box_a,box_b):
    min_xy = torch.max(box_a[:,None,:2],box_b[None,:,:2])
    max_xy = torch.min(box_a[:,None,2:],box_b[None,:,2:])
    inter = torch.clamp(max_xy-min_xy,min=0)
    return inter[:,:,0] * inter[:,:,1]
def get_size(box):
    return (box[:,2]-box[:,0]) * (box[:,3] - box[:,1])
def jaccard(box_a,box_b):
    inter = intersection(box_a,box_b)
    union = get_size(box_a).unsqueeze(1) + get_size(box_b).unsqueeze(0) - inter
    return inter/union


### ssd_loss ###
def get_y(bbox,clas):
    bbox = bbox.view(-1,4)
    bb_keep = ((bbox[:,2] - bbox[:,0])>0.).nonzero()[:,0]
    return bbox[bb_keep], clas[bb_keep]
    
def actn_to_bb(actn, anchors):
    actn_bbs = torch.tanh(actn)
    actn_ctrs = (actn_bbs.cuda()[:,:2] * grid_sizes.cuda()/2) + anchors.cuda()[:,:2]
    actn_hw = (1 + actn_bbs.cuda()[:,2:]/2) * anchors.cuda()[:,2:]
    return hw2corners(actn_ctrs,actn_hw)

def map_to_ground_truth(overlaps, print_it=False):
    prior_overlap, prior_idx = overlaps.max(1)
    if print_it: print(prior_overlap)
    gt_overlap, gt_idx = overlaps.max(0)
    gt_overlap[prior_idx] = 1.99
    for i,o in enumerate(prior_idx): gt_idx[o] = i
    return gt_overlap,gt_idx

def ssd_1_loss(b_bb, b_c, bbox, clas, print_it=False, use_ab=True):
    bbox,clas = get_y(bbox,clas)
    a_ic = actn_to_bb(b_bb, anchors)
    overlaps = jaccard(bbox.data, (anchor_cnr.cuda() if use_ab else a_ic).data)
    gt_overlap,gt_idx = map_to_ground_truth(overlaps)
    gt_clas = clas[gt_idx]
    pos = gt_overlap > 0.4
    pos_idx = torch.nonzero(pos)[:,0]
    gt_clas[~pos] = dls.ncls
    gt_bbox = bbox[gt_idx]
    loc_loss = ((a_ic[pos_idx] - gt_bbox[pos_idx]).abs()).mean()
    clas_loss  = loss_f(b_c, gt_clas)
    return loc_loss, clas_loss

def ssd_loss(acts, bbs, lbls, print_it=False):
    bb_sum, lbl_sum = 0., 0.
    for abb, albl, bb, lbl in zip(*acts, bbs, lbls):
        bb_loss, lbl_loss = ssd_1_loss(abb, albl, bb, lbl, print_it)
        bb_sum += bb_loss
        lbl_sum += lbl_loss
    met.bb.append(bb_sum)
    met.lbl.append(lbl_sum)
    return bb_sum + lbl_sum

In [ ]:
class ShowLosses(Metric):
    
    def __init__(self): self.bb,self.lbl = [],[]
        
    def reset(self):    self.bb,self.lbl = [],[] 
        
    def accumulate(self, learn): pass
    
    @property
    def value(self):
        bb_loss  = round((sum(self.bb )/len(self.bb )).item(), 2)
        lbl_loss = round((sum(self.lbl)/len(self.lbl)).item(), 2)
        return bb_loss, lbl_loss
    
    @property
    def name(self): return "MAE, Focal"

# Train

## With scaling preds

In [ ]:
# device='cuda'
# size = im_sz
# body = create_body(resnet34, pretrained=True)
# head = SSD_MultiHead(k, -4.)
# mod = CustMod(body, head)
# loss_f = FocalLoss(dls.ncls, device)

In [ ]:
# learner = Learner(dls.cuda(), mod, loss_func=ssd_loss)
# learner.freeze()

In [ ]:
# learner.lr_find()

In [ ]:
# learner.fit_one_cycle(8, lr=1e-3)

## Without scaling preds

In [ ]:
device='cuda'
size = im_sz
body = create_body(resnet34, pretrained=True)
head = SSD_MultiHead(k)
mod = CustMod(body, head)
loss_f = FocalLoss(dls.ncls, device)
met = ShowLosses()

In [ ]:
learner = Learner(dls.cuda(), mod, loss_func=ssd_loss, metrics=met)
learner.freeze()

In [ ]:
# learner.lr_find()

In [ ]:
learner.fit_one_cycle(6, lr=1e-3)

epoch,train_loss,valid_loss,"MAE, Focal",time
0,37.807579,32.324768,"(21.07, 11.17)",00:11
1,33.459534,134.450790,"(36.67, 97.53)",00:11
2,32.157589,29.494228,"(19.59, 9.82)",00:11
3,31.064367,102.948166,"(23.06, 79.31)",00:11
4,30.416964,27.452950,"(19.23, 8.14)",00:11
5,29.479332,27.339811,"(19.28, 7.98)",00:11


# Stepping Through a Batch

Shared

In [ ]:
body = create_body(resnet34, pretrained=True)
b_idx = 0

## JH

### Init

In [ ]:
# Init
size=im_sz

In [ ]:
anchors

tensor([[0.1250, 0.1250, 0.1875, 0.1875],
        [0.1250, 0.1250, 0.1875, 0.0938],
        [0.1250, 0.1250, 0.0938, 0.1875],
        [0.1250, 0.1250, 0.2500, 0.2500],
        [0.1250, 0.1250, 0.2500, 0.1250],
        [0.1250, 0.1250, 0.1250, 0.2500],
        [0.1250, 0.1250, 0.3250, 0.3250],
        [0.1250, 0.1250, 0.3250, 0.1625],
        [0.1250, 0.1250, 0.1625, 0.3250],
        [0.3750, 0.1250, 0.1875, 0.1875],
        [0.3750, 0.1250, 0.1875, 0.0938],
        [0.3750, 0.1250, 0.0938, 0.1875],
        [0.3750, 0.1250, 0.2500, 0.2500],
        [0.3750, 0.1250, 0.2500, 0.1250],
        [0.3750, 0.1250, 0.1250, 0.2500],
        [0.3750, 0.1250, 0.3250, 0.3250],
        [0.3750, 0.1250, 0.3250, 0.1625],
        [0.3750, 0.1250, 0.1625, 0.3250],
        [0.6250, 0.1250, 0.1875, 0.1875],
        [0.6250, 0.1250, 0.1875, 0.0938],
        [0.6250, 0.1250, 0.0938, 0.1875],
        [0.6250, 0.1250, 0.2500, 0.2500],
        [0.6250, 0.1250, 0.2500, 0.1250],
        [0.6250, 0.1250, 0.1250, 0

### Get batch, acts, item

In [ ]:
batch = next(iter(dls.cpu().valid))
acts = mod(batch[0])
bbs, lbls = batch[1], batch[2]

In [ ]:
# Get acts and targs for a single im
abb,albl,bb,lbl = list(zip(*acts,bbs,lbls))[b_idx]

In [ ]:
abb.shape,albl.shape

(torch.Size([189, 4]), torch.Size([189, 21]))

In [ ]:
bb,lbl

(tensor([[-0.9177, -0.7167,  0.5527,  0.9417],
         [-0.5219, -0.7125,  0.2134,  0.1583],
         [ 0.0437,  0.1958,  0.5321,  0.7083],
         [-0.0026,  0.1167,  0.3728,  0.4792],
         [ 0.0000,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  0.0000]]),
 tensor([12, 14, 12, 14,  0,  0,  0,  0,  0,  0,  0,  0]))

### ssd item loss

In [ ]:
# Process single im acts and targs in loss fxn
#assignments
b_bb,b_c,bbox,clas=abb,albl,bb,lbl
print_it=False
use_ab=True

In [ ]:
bbox,clas = get_y(bbox,clas)
bbox,clas

(tensor([[-0.9177, -0.7167,  0.5527,  0.9417],
         [-0.5219, -0.7125,  0.2134,  0.1583],
         [ 0.0437,  0.1958,  0.5321,  0.7083],
         [-0.0026,  0.1167,  0.3728,  0.4792]]),
 tensor([12, 14, 12, 14]))

In [ ]:
a_ic = actn_to_bb(b_bb, anchors)
a_ic.shape, a_ic[0:5]

(torch.Size([189, 4]),
 tensor([[-0.8334, -0.9761, -0.3795, -0.5666],
         [-0.7941, -0.9038, -0.4184, -0.7025],
         [-0.8372, -0.7897, -0.6151, -0.5645],
         [-1.1843, -0.9008, -0.7673, -0.4571],
         [-0.9295, -1.0600, -0.3284, -0.8027]], device='cuda:0',
        grad_fn=<SliceBackward>))

In [ ]:
overlaps = jaccard(bbox.data, anchor_cnr.data)
overlaps.shape, overlaps

(torch.Size([4, 189]),
 tensor([[1.4417e-02, 7.2087e-03, 7.2087e-03, 2.5631e-02, 1.2815e-02, 1.2815e-02,
          4.3316e-02, 2.1658e-02, 2.1658e-02, 1.4417e-02, 7.2087e-03, 7.2087e-03,
          2.5631e-02, 1.2815e-02, 1.2815e-02, 4.3316e-02, 2.1658e-02, 2.1658e-02,
          1.6285e-03, 8.1941e-04, 0.0000e+00, 5.2958e-03, 2.6744e-03, 0.0000e+00,
          1.1657e-02, 5.9183e-03, 1.1688e-03, 0.0000e+00, 0.0000e+00, 0.0000e+00,
          0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
          1.4417e-02, 7.2087e-03, 7.2087e-03, 2.5631e-02, 1.2815e-02, 1.2815e-02,
          4.3316e-02, 2.1658e-02, 2.1658e-02, 1.4417e-02, 7.2087e-03, 7.2087e-03,
          2.5631e-02, 1.2815e-02, 1.2815e-02, 4.3316e-02, 2.1658e-02, 2.1658e-02,
          1.6285e-03, 8.1941e-04, 0.0000e+00, 5.2958e-03, 2.6744e-03, 0.0000e+00,
          1.1657e-02, 5.9183e-03, 1.1688e-03, 0.0000e+00, 0.0000e+00, 0.0000e+00,
          0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.000

In [ ]:
gt_overlap,gt_idx = map_to_ground_truth(overlaps)
gt_overlap,gt_idx

(tensor([1.2587e-01, 7.2247e-02, 6.6432e-02, 2.0175e-01, 1.1837e-01, 1.1064e-01,
         1.9900e+00, 1.5954e-01, 1.7231e-01, 5.7705e-02, 3.3999e-02, 3.0558e-02,
         8.9835e-02, 5.4819e-02, 5.0439e-02, 1.2775e-01, 8.2263e-02, 7.6957e-02,
         1.6285e-03, 8.1941e-04, 0.0000e+00, 5.5953e-03, 2.6744e-03, 0.0000e+00,
         1.8260e-02, 5.9183e-03, 1.1688e-03, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         2.5840e-01, 1.2920e-01, 1.2920e-01, 4.0557e-01, 2.2969e-01, 2.0659e-01,
         4.7072e-01, 3.3258e-01, 2.9776e-01, 1.4044e-01, 7.0222e-02, 7.0222e-02,
         2.4968e-01, 1.2484e-01, 1.2484e-01, 4.1212e-01, 2.0677e-01, 2.1098e-01,
         1.6285e-03, 8.1941e-04, 0.0000e+00, 2.6357e-02, 1.4472e-02, 0.0000e+00,
         6.7897e-02, 3.8776e-02, 1.1688e-03, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         1.2181e-01, 6.5254e

In [ ]:
gt_clas = clas[gt_idx]
gt_clas

tensor([14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14,
        12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12,
        14, 14, 14, 14, 14, 14, 14, 14, 14, 12, 12, 12, 12, 12, 12, 12, 12, 12,
        12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12,
        12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12,
        12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12,
        12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12,
        12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12,
        14, 14, 14, 14, 14, 14, 12, 14, 14, 12, 12, 12, 12, 12, 12, 12, 12, 12,
        12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12,
        12, 12, 12, 12, 12, 12, 12, 12, 12])

In [ ]:
pos = gt_overlap > 0.4
pos

tensor([False, False, False, False, False, False,  True, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,  True,
        False, False,  True, False, False, False, False, False, False, False,
        False,  True, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, 

In [ ]:
pos_idx = torch.nonzero(pos)[:,0]
pos_idx

tensor([  6,  39,  42,  51, 144, 146, 147, 148, 149, 151, 152, 181, 184, 186])

In [ ]:
gt_clas[~pos] = dls.ncls
gt_clas

tensor([20, 20, 20, 20, 20, 20, 14, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20,
        20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20,
        20, 20, 20, 14, 20, 20, 14, 20, 20, 20, 20, 20, 20, 20, 20, 12, 20, 20,
        20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20,
        20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20,
        20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20,
        20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20,
        20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20,
        14, 20, 14, 14, 14, 14, 20, 14, 14, 20, 20, 20, 20, 20, 20, 20, 20, 20,
        20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20,
        20, 12, 20, 20, 12, 20, 12, 20, 20])

In [ ]:
gt_bbox = bbox[gt_idx]
gt_bbox.shape, gt_bbox

(torch.Size([189, 4]),
 tensor([[-0.0026,  0.1167,  0.3728,  0.4792],
         [-0.0026,  0.1167,  0.3728,  0.4792],
         [-0.0026,  0.1167,  0.3728,  0.4792],
         [-0.0026,  0.1167,  0.3728,  0.4792],
         [-0.0026,  0.1167,  0.3728,  0.4792],
         [-0.0026,  0.1167,  0.3728,  0.4792],
         [-0.5219, -0.7125,  0.2134,  0.1583],
         [-0.0026,  0.1167,  0.3728,  0.4792],
         [-0.0026,  0.1167,  0.3728,  0.4792],
         [-0.0026,  0.1167,  0.3728,  0.4792],
         [-0.0026,  0.1167,  0.3728,  0.4792],
         [-0.0026,  0.1167,  0.3728,  0.4792],
         [-0.0026,  0.1167,  0.3728,  0.4792],
         [-0.0026,  0.1167,  0.3728,  0.4792],
         [-0.0026,  0.1167,  0.3728,  0.4792],
         [-0.0026,  0.1167,  0.3728,  0.4792],
         [-0.0026,  0.1167,  0.3728,  0.4792],
         [-0.0026,  0.1167,  0.3728,  0.4792],
         [-0.9177, -0.7167,  0.5527,  0.9417],
         [-0.9177, -0.7167,  0.5527,  0.9417],
         [-0.9177, -0.7167,  0.5527, 

In [ ]:
loc_loss = ((a_ic.cpu()[pos_idx] - gt_bbox[pos_idx]).abs()).mean()
loc_loss

tensor(0.6299, grad_fn=<MeanBackward0>)

In [ ]:
device='cpu'
clas_loss  = loss_f(b_c.cpu(), gt_clas.cpu())
clas_loss

tensor(0.6595, grad_fn=<DivBackward0>)

### lbl loss

In [ ]:
# clas_loss
#assignments
preds,targets = b_c.cpu(),gt_clas.cpu()
num_classes=20

In [ ]:
t = one_hot_embedding(targets, num_classes+1)
t.shape, t

(torch.Size([189, 21]),
 tensor([[0., 0., 0.,  ..., 0., 0., 1.],
         [0., 0., 0.,  ..., 0., 0., 1.],
         [0., 0., 0.,  ..., 0., 0., 1.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 1.],
         [0., 0., 0.,  ..., 0., 0., 1.],
         [0., 0., 0.,  ..., 0., 0., 1.]]))

In [ ]:
t = tensor(t[:,:-1].contiguous())
t.shape, t

(torch.Size([189, 20]),
 tensor([[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]]))

In [ ]:
x = preds[:,:-1]
x.shape,x

(torch.Size([189, 20]),
 tensor([[-4.0802, -3.6891, -3.8384,  ..., -3.7442, -3.4973, -3.8569],
         [-3.3145, -3.9785, -4.4999,  ..., -4.2666, -4.1959, -3.7895],
         [-3.8018, -4.3627, -4.3670,  ..., -4.2720, -3.6047, -3.8739],
         ...,
         [-4.0017, -3.9917, -4.0002,  ..., -3.9941, -3.9885, -3.9979],
         [-4.0118, -3.9898, -3.9929,  ..., -4.0014, -4.0001, -3.9908],
         [-3.9951, -4.0137, -4.0054,  ..., -3.9829, -3.9871, -3.9828]],
        grad_fn=<SliceBackward>))

In [ ]:
def get_weight(x,t):
    alpha,gamma = 0.25,2.
    p = x.sigmoid()
    pt = p*t + (1-p)*(1-t)
    w = alpha*t + (1-alpha)*(1-t)
    return w * (1-pt).pow(gamma)

In [ ]:
w = get_weight(x,t).detach()
w.shape, w

(torch.Size([189, 20]),
 tensor([[2.0725e-04, 4.4598e-04, 3.3307e-04,  ..., 4.0050e-04, 6.4780e-04,
          3.2122e-04],
         [9.2278e-04, 2.5309e-04, 9.0553e-05,  ..., 1.4356e-04, 1.6502e-04,
          3.6657e-04],
         [3.5785e-04, 1.1875e-04, 1.1775e-04,  ..., 1.4205e-04, 5.2567e-04,
          3.1073e-04],
         ...,
         [2.4181e-04, 2.4661e-04, 2.4252e-04,  ..., 2.4544e-04, 2.4818e-04,
          2.4365e-04],
         [2.3708e-04, 2.4751e-04, 2.4603e-04,  ..., 2.4197e-04, 2.4256e-04,
          2.4704e-04],
         [2.4495e-04, 2.3618e-04, 2.4009e-04,  ..., 2.5093e-04, 2.4887e-04,
          2.5096e-04]]))